In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
import xgboost as xg
from sklearn.svm import SVR
import numpy as np
import sklearn
import tensorflow as tf
from sklearn.model_selection import KFold               

In [72]:
data=pd.read_csv(r"/home/sagar/Downloads/Train.csv")

In [66]:
submission=pd.read_csv(r"/home/sagar/Downloads/submissionZindi.csv")

In [ ]:
submission['target_2019'] = model2.predict(x_test)

In [68]:
submission.to_csv(r"/home/sagar/Downloads/meta_xgb.csv",index=False)


In [73]:
precip_features_2019 = []
precip_features_2015 = []
for col in data.columns:
  if '2019' in col:
    precip_features_2019.append(col)
  elif 'precip 2014' in col:
    precip_features_2015.append(col)
  elif 'precip 2015' in col:
    precip_features_2015.append(col)

In [39]:
train = data[data.columns.difference(precip_features_2019)]
x_train=train.drop(['target_2015','Square_ID'],axis=1)
y_train=train['target_2015']

In [74]:
precip_features_2019.extend(['X','Y','elevation', 'LC_Type1_mode','Square_ID'])
test = data[precip_features_2019]
x_test=test.drop("Square_ID",axis=1)


In [75]:
new_2015_cols = {}
for col, number in zip(precip_features_2015, range(1, len(precip_features_2015) + 1)):
  if 'precip' in col:
    new_2015_cols[col] = 'week_' + str(number) + '_precip'

    
# Creating a dictionary of column names to be renamed for the testing dataset
#
new_2019_cols = {}
for col, number in zip(precip_features_2019, range(1, len(precip_features_2019) + 1)):
  if 'precip' in col:
    new_2019_cols[col] = 'week_' + str(number) + '_precip'
    
# Renaming the columns
#
train.rename(columns = new_2015_cols, inplace = True)
test.rename(columns = new_2019_cols, inplace = True)                      

In [76]:
x_train=train.drop(['target_2015','Square_ID'],axis=1)
y_train=train['target_2015']
x_test=test.drop("Square_ID",axis=1)

In [77]:
x_test.shape

(16466, 21)

In [59]:
model1=RandomForestRegressor(n_estimators=100)
model1.fit(x_train,y_train)

model2=xg.XGBRegressor(n_estimators=900)
model2.fit(x_train,y_train)


model3=AdaBoostRegressor()
model3.fit(x_train,y_train)

model4=GradientBoostingRegressor()
model4.fit(x_train,y_train)

model5=BaggingRegressor()
model5.fit(x_train,y_train)

model6=SVR()
model6.fit(x_train,y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [60]:
data_x=pd.DataFrame()
x=0
rftl,bagl,adal,data_y,svrl=list(),list(),list(),list(),list()
kfold= KFold(n_splits=5,shuffle=True)
for train_ix,test_ix in kfold.split(x_train):
    X_train,X_test=x_train.iloc[train_ix,:],x_train.iloc[test_ix,:]
    Y_train,Y_test=y_train.iloc[train_ix],y_train.iloc[test_ix]
    data_y.extend(Y_test)
    data_x=data_x.append(X_test)

    rft=RandomForestRegressor(n_estimators=100)
    rft.fit(X_train,Y_train)
    rftl.extend(rft.predict(X_test))
    
    ada=AdaBoostRegressor()
    ada.fit(x_train,y_train)
    adal.extend(ada.predict(X_test))

   
    bag=BaggingRegressor()
    bag.fit(X_train,Y_train)
    bagl.extend(bag.predict(X_test))
    
    svr=SVR()
    svr.fit(X_train,Y_train)
    svrl.extend(svr.predict(X_test))
    
    x=x+1
    print(x)

1
2
3
4
5


In [11]:
#add the data from base learner to the input
print(len(data_y))
print(data_x.shape)

16466
(16466, 21)


In [61]:
meta_data= data_x
meta_data.insert(21,'rft',rftl)
meta_data.insert(22,'ada',adal)
meta_data.insert(23,'svr',svrl)
meta_data.insert(24,'bag',bagl)


In [62]:
meta_model=xg.XGBRegressor(n_estimators=900)
meta_model.fit(meta_data,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [70]:
import joblib
filename='meta_xgb_r_a_b_s'
joblib.dump(meta_model,filename)

['meta_xgb_r_a_b_s']

In [63]:
sub_1=model1.predict(x_test)
sub_3=model3.predict(x_test)
sub_5=model5.predict(x_test)
sub_6=model6.predict(x_test)
meta_data_test=x_test
meta_data_test.insert(21,'rft',sub_1)
meta_data_test.insert(22,'ada',sub_3)
meta_data_test.insert(23,'svr',sub_5)
meta_data_test.insert(24,'bag',sub_6)
meta_data_test=meta_data_test[meta_data.columns]
meta_pred=meta_model.predict(meta_data_test)



In [65]:
len(meta_pred)

16466

In [14]:
model1=RandomForestRegressor(n_estimators=100)
model1.fit(x_data,y_data)
rmse=sklearn.metrics.mean_squared_error
print('random forest:',np.sqrt(rmse(y_data,model1.predict(x_data))))


('random forest:', 0.03316363093261265)


In [16]:
print('random forest:    ',rmse(y_data,model1.predict(x_data)))


('random forest:    ', 0.001099826416634543)


In [8]:
rmse=sklearn.metrics.mean_squared_error
print('random forest:',np.sqrt(rmse(y_test,model1.predict(x_test))))
print('xgb',np.sqrt(rmse(y_test,model2.predict(x_test))))
print('adaboost',np.sqrt(rmse(y_test,model3.predict(x_test))))
print('GradientBoosting',np.sqrt(rmse(y_test,model4.predict(x_test))))
print('Bagging',np.sqrt(rmse(y_test,model5.predict(x_test))))


('random forest:', 0.09728735012487937)
('xgb', 0.11356744755495865)
('adaboost', 0.17793326372289142)
('GradientBoosting', 0.12755603053454412)
('Bagging', 0.10087138902175086)


In [41]:
print('random forest:    ',rmse(y_test,model1.predict(x_test)))
print('xgb:              ',rmse(y_test,model2.predict(x_test)))
print('adaboost:         ',rmse(y_test,model3.predict(x_test)))
print('gradient boosting:',rmse(y_test,model4.predict(x_test)))
print('bagging:          ',rmse(y_test,model5.predict(x_test)))


('random forest:    ', 0.009464828494320865)
('xgb:              ', 0.012897565144148283)
('adaboost:         ', 0.031660246339080024)
('gradient boosting:', 0.016270540925729552)
('bagging:          ', 0.010175037123177398)


In [ ]:
data_x

In [26]:
ml=pd.DataFrame()
ml.insert(0,"Square_ID",data['Square_ID'])
ml.insert(1,"target_2019",model1.predict(x_data))
ml.to_csv(r"/home/sagar/Downloads/sub.csv")